# 에이전트를 구축하십시오

Langchain은 [에이전트] (/docs/concepts/agents) 또는 [LLMS] (/docs/concepts/chat_models)를 사용하는 시스템의 생성을 지원하여 엔진을 추론하여 동작을 수행하는 데 필요한 작업 및 입력을 결정합니다.
작업을 실행 한 후 결과를 LLM으로 Fed Back으로 공급하여 더 많은 작업이 필요한지 또는 완료하는 것이 괜찮은지 여부를 결정할 수 있습니다.이것은 종종 [Tool-Calling] (/docs/concepts/tool_calling)를 통해 달성됩니다.

이 튜토리얼에서는 검색 엔진과 상호 작용할 수있는 에이전트를 구축합니다.이 에이전트 질문을하고 검색 도구에 전화를 걸고 대화를 할 수 있습니다.

## 엔드 투 엔드 에이전트

아래 코드 스 니펫은 LLM을 사용하여 사용할 도구를 결정하는 완벽한 기능 에이전트를 나타냅니다.일반 검색 도구가 장착되어 있습니다.대화 메모리가 있습니다. 즉, 다중 전환 챗봇으로 사용할 수 있음을 의미합니다.

가이드의 나머지 부분에서는 개별 구성 요소와 각 부분이하는 일을 안내하지만 코드를 잡고 시작하려면 자유롭게 사용하십시오!


In [1]:
# Import relevant functionality
from langchain.chat_models import init_chat_model
from langchain_tavily import TavilySearch
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

# Create the agent
memory = MemorySaver()
model = init_chat_model("anthropic:claude-3-5-sonnet-latest")
search = TavilySearch(max_results=2)
tools = [search]
agent_executor = create_react_agent(model, tools, checkpointer=memory)

In [2]:
# Use the agent
config = {"configurable": {"thread_id": "abc123"}}

input_message = {
    "role": "user",
    "content": "Hi, I'm Bob and I live in SF.",
}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Hi, I'm Bob and I live in SF.
================================== Ai Message ==================================

Hello Bob! I notice you've introduced yourself and mentioned you live in SF (San Francisco), but you haven't asked a specific question or made a request that requires the use of any tools. Is there something specific you'd like to know about San Francisco or any other topic? I'd be happy to help you find information using the available search tools.


In [3]:
input_message = {
    "role": "user",
    "content": "What's the weather where I live?",
}

for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What's the weather where I live?
================================== Ai Message ==================================

[{'text': 'Let me search for current weather information in San Francisco.', 'type': 'text'}, {'id': 'toolu_011kSdheoJp8THURoLmeLtZo', 'input': {'query': 'current weather San Francisco CA'}, 'name': 'tavily_search', 'type': 'tool_use'}]
Tool Calls:
  tavily_search (toolu_011kSdheoJp8THURoLmeLtZo)
 Call ID: toolu_011kSdheoJp8THURoLmeLtZo
  Args:
    query: current weather San Francisco CA
================================= Tool Message =================================
Name: tavily_search

{"query": "current weather San Francisco CA", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "Weather in San Francisco, CA", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America',

## 설정

### Jupyter 노트

이 안내서 (및 문서의 다른 가이드 대부분)는 [Jupyter Notebooks] (https://jupyter.org/)를 사용하고 독자도 마찬가지라고 가정합니다.Jupyter 노트북은 LLM 시스템을 사용하는 방법을 배우기위한 완벽한 대화식 환경입니다. 왜냐하면 종종 일이 잘못 될 수 있기 때문에 (예기치 않은 출력, API 다운 등), 이러한 사례를 관찰하는 것은 LLM으로 건축을 더 잘 이해하는 좋은 방법입니다.

이 튜토리얼과 다른 튜토리얼은 아마도 Jupyter 노트북에서 가장 편리하게 실행됩니다.설치 방법에 대한 지침은 [여기] (https://jupyter.org/install)을 참조하십시오.

### 설치

Langchain Run을 설치하려면 :


In [ ]:
%pip install -U langgraph langchain-tavily langgraph-checkpoint-sqlite

자세한 내용은 [설치 안내서] (/docs/how_to/installation)를 참조하십시오.

### Langsmith

Langchain으로 구축 한 많은 응용 프로그램에는 LLM 통화가 여러 번 발생하여 여러 단계가 포함됩니다.
이러한 응용 프로그램이 점점 더 복잡해지면 체인이나 에이전트 내에서 정확히 무슨 일이 일어나고 있는지 검사하는 것이 중요합니다.
이를 수행하는 가장 좋은 방법은 [langsmith] (https://smith.langchain.com)입니다.

위의 링크에 가입 한 후에는 환경 변수를 로깅 추적을 시작하도록 설정하십시오.

```쉘
내보내기 langsmith_tracing = "true"
내보내기 langsmith_api_key = "..."
```

또는 노트북에 있으면 다음과 같이 설정할 수 있습니다.

```Python
getpass 가져 오기
OS 가져 오기

os.environ [ "langsmith_tracing"] = "true"
os.environ [ "langsmith_api_key"] = getpass.getpass ()
```

### tavily

우리는 [tavily] (/docs/integrations/tools/tavily_search) (검색 엔진)을 도구로 사용할 것입니다.
이를 사용하려면 API 키를 가져 와서 설정해야합니다.

```Bash
tavily_api_key = "..."내보내기
```

또는 노트북에 있으면 다음과 같이 설정할 수 있습니다.

```Python
getpass 가져 오기
OS 가져 오기

os.environ [ "tavily_api_key"] = getpass.getpass ()
```


## 도구 정의

먼저 사용하려는 도구를 만들어야합니다.우리의 주요 선택 도구는 검색 엔진 인 [tavily] (/docs/integrations/tools/tavily_search)입니다.우리는 전용 [langchain-tavily] (https://pypi.org/project/langchain-tavily/) [통합 패키지] (/docs/concepts/architecture/#통합-패키지)을 Langchain과 함께 도구로 쉽게 사용할 수 있습니다.


In [5]:
from langchain_tavily import TavilySearch

search = TavilySearch(max_results=2)
search_results = search.invoke("What is the weather in SF")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

{'query': 'What is the weather in SF', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'Weather in San Francisco, CA', 'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1750168606, 'localtime': '2025-06-17 06:56'}, 'current': {'last_updated_epoch': 1750167900, 'last_updated': '2025-06-17 06:45', 'temp_c': 11.7, 'temp_f': 53.1, 'is_day': 1, 'condition': {'text': 'Fog', 'icon': '//cdn.weatherapi.com/weather/64x64/day/248.png', 'code': 1135}, 'wind_mph': 4.0, 'wind_kph': 6.5, 'wind_degree': 215, 'wind_dir': 'SW', 'pressure_mb': 1017.0, 'pressure_in': 30.02, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 86, 'cloud': 0, 'feelslike_c': 11.3, 'feelslike_f': 52.4, 'windchill_c': 8.7, 'windchill_f': 47.7, 'heatindex_c': 9.8, 'heatindex_f': 49.7, 'dewpoint_c': 9.6, 'dewpoint_f':

:::팁

많은 응용 분야에서 사용자 정의 도구를 정의 할 수 있습니다.Langchain은 관습을 지원합니다
파이썬 기능 및 기타 수단을 통한 도구 생성.참조
[도구를 만드는 방법] (/docs/how_to/custom_tools/) 자세한 내용은 안내서.

:::


## 언어 모델 사용

다음으로 언어 모델을 사용하여 도구를 호출하는 방법을 배우겠습니다.Langchain은 상호 교환 할 수있는 다양한 언어 모델을 지원합니다. 아래에서 사용할 것을 선택하십시오!

"@테마/chatmodeltabs"에서 chatmodeltabs 가져 오기;

<chatmodeltabs atedrideparams = {{openai : {model : "gpt-4.1"}} />


In [6]:
# | output: false
# | echo: false

from langchain_anthropic import ChatAnthropic

model = ChatAnthropic(model="claude-3-5-sonnet-latest")

메시지 목록을 전달하여 언어 모델을 호출 할 수 있습니다.기본적으로 응답은 'content'문자열입니다.


In [7]:
query = "Hi!"
response = model.invoke([{"role": "user", "content": query}])
response.text()

'Hello! How can I help you today?'

이제이 모델이 도구 호출을 수행 할 수 있도록하는 것이 어떤지 확인할 수 있습니다.우리가`.bind_tools`를 사용하여 이러한 도구에 대한 언어 모델 지식을 제공 할 수 있도록


In [8]:
model_with_tools = model.bind_tools(tools)

이제 모델을 호출 할 수 있습니다.먼저 일반 메시지로 전화하여 어떻게 응답하는지 살펴 보겠습니다.우리는 'Content'필드와 '도구 _calls'필드를 모두 볼 수 있습니다.


In [11]:
query = "Hi!"
response = model_with_tools.invoke([{"role": "user", "content": query}])

print(f"Message content: {response.text()}\n")
print(f"Tool calls: {response.tool_calls}")

Message content: Hello! I'm here to help you. I have access to a powerful search tool that can help answer questions and find information about various topics. What would you like to know about?

Feel free to ask any question or request information, and I'll do my best to assist you using the available tools.

Tool calls: []


이제 도구를 호출 할 것으로 예상되는 입력으로 호출해 봅시다.


In [16]:
query = "Search for the weather in SF"
response = model_with_tools.invoke([{"role": "user", "content": query}])

print(f"Message content: {response.text()}\n")
print(f"Tool calls: {response.tool_calls}")

Message content: I'll help you search for information about the weather in San Francisco.

Tool calls: [{'name': 'tavily_search', 'args': {'query': 'current weather San Francisco'}, 'id': 'toolu_015gdPn1jbB2Z21DmN2RAnti', 'type': 'tool_call'}]


우리는 이제 텍스트 내용이 없지만 공구 통화가 있음을 알 수 있습니다!그것은 우리가 Tavily 검색 도구를 호출하기를 원합니다.

이것은 아직 그 도구를 부르는 것이 아닙니다. 단지 우리에게 말하고 있습니다.실제로 그것을 부르려면 에이전트를 만들고 싶습니다.


## 에이전트를 만듭니다

도구와 LLM을 정의 했으므로 에이전트를 만들 수 있습니다.우리는 [langgraph] (/docs/concepts/architecture/#langgraph)를 사용하여 에이전트를 구성 할 것입니다.
현재, 우리는 에이전트를 구성하기 위해 높은 수준의 인터페이스를 사용하고 있지만, Langgraph의 좋은 점은이 높은 수준의 인터페이스가 에이전트 로직을 수정하려는 경우 저수준의 제어 가능한 API로 뒷받침된다는 것입니다.


이제 LLM과 도구로 에이전트를 초기화 할 수 있습니다.

`model_with_tools`가 아닌 'model'을 통과하고 있습니다.`reation_react_agent`는 후드 아래에서 우리를 위해`.bind_tools`를 호출하기 때문입니다.


In [13]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

## 에이전트를 실행하십시오

이제 몇 가지 쿼리로 에이전트를 실행할 수 있습니다!현재로서는 모두 ** 무국적 ** 쿼리입니다 (이전 상호 작용을 기억하지 않습니다).에이전트는 상호 작용이 끝날 때 ** Final ** 상태를 반환합니다 (모든 입력을 포함하여 나중에 출력 만 얻는 방법에 대해 볼 수 있습니다).

먼저, 도구를 호출 할 필요가 없을 때 어떻게 응답하는지 봅시다.


In [14]:
input_message = {"role": "user", "content": "Hi!"}
response = agent_executor.invoke({"messages": [input_message]})

for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

Hi!
================================== Ai Message ==================================

Hello! I'm here to help you with your questions using the available search tools. Please feel free to ask any question, and I'll do my best to find relevant and accurate information for you.


후드 아래에서 무슨 일이 일어나고 있는지 (그리고 도구를 부르지 않도록) [Langsmith Trace] (https://smith.langchain.com/public/28311faa-e135-4d6a-ab6b-caecf6482aaa/r)를 살펴볼 수 있습니다.

이제 도구를 호출 해야하는 예를 들어 보겠습니다.


In [15]:
input_message = {"role": "user", "content": "Search for the weather in SF"}
response = agent_executor.invoke({"messages": [input_message]})

for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

Search for the weather in SF
================================== Ai Message ==================================

[{'text': "I'll help you search for weather information in San Francisco. Let me use the search engine to find current weather conditions.", 'type': 'text'}, {'id': 'toolu_01WWcXGnArosybujpKzdmARZ', 'input': {'query': 'current weather San Francisco SF'}, 'name': 'tavily_search', 'type': 'tool_use'}]
Tool Calls:
  tavily_search (toolu_01WWcXGnArosybujpKzdmARZ)
 Call ID: toolu_01WWcXGnArosybujpKzdmARZ
  Args:
    query: current weather San Francisco SF
================================= Tool Message =================================
Name: tavily_search

{"query": "current weather San Francisco SF", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "Weather in San Francisco, CA", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'San Francisco', 'r

[Langsmith Trace] (https://smith.langchain.com/public/f520839d-cd4d-4495-8764-e32b548e235d/r)를 검색 도구를 효과적으로 호출 할 수 있습니다.


## 스트리밍 메시지

우리는 최종 응답을 얻기 위해`.invoke`를 사용하여 에이전트를 어떻게 호출 할 수 있는지 보았습니다.에이전트가 여러 단계를 수행하면 시간이 걸릴 수 있습니다.중간 진행 상황을 보여주기 위해 메시지가 발생할 때 되돌아 갈 수 있습니다.


In [17]:
for step in agent_executor.stream({"messages": [input_message]}, stream_mode="values"):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Search for the weather in SF
================================== Ai Message ==================================

[{'text': "I'll help you search for information about the weather in San Francisco.", 'type': 'text'}, {'id': 'toolu_01DCPnJES53Fcr7YWnZ47kDG', 'input': {'query': 'current weather San Francisco'}, 'name': 'tavily_search', 'type': 'tool_use'}]
Tool Calls:
  tavily_search (toolu_01DCPnJES53Fcr7YWnZ47kDG)
 Call ID: toolu_01DCPnJES53Fcr7YWnZ47kDG
  Args:
    query: current weather San Francisco
================================= Tool Message =================================
Name: tavily_search

{"query": "current weather San Francisco", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "Weather in San Francisco", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 

## 스트리밍 토큰

후면 메시지를 스트리밍하는 것 외에도 다시 토큰을 스트리밍하는 것이 유용합니다.
`stream_mode = "message"`을 지정하여이를 수행 할 수 있습니다.


::: 메모

아래에서는`message.text ()`을 사용하며`langchain-core> = 0.3.37`가 필요합니다.

:::


In [18]:
for step, metadata in agent_executor.stream(
    {"messages": [input_message]}, stream_mode="messages"
):
    if metadata["langgraph_node"] == "agent" and (text := step.text()):
        print(text, end="|")

I|'ll help you search for information| about the weather in San Francisco.|Base|d on the search results, here|'s the current weather in| San Francisco:
-| Temperature: 53.1°F (|11.7°C)
-| Condition: Foggy
- Wind:| 4.0 mph from| the Southwest
- Humidity|: 86%|
- Visibility: 9|.0 miles
- Pressure: |30.02 in|Hg

The weather| is characteristic of San Francisco, with| foggy conditions and mild temperatures|. The "feels like" temperature is slightly| lower at 52.4|°F (11.|3°C)| due to the wind chill effect|.|

## 메모리 추가

앞에서 언급 했듯이이 에이전트는 상태가 없습니다.이것은 이전 상호 작용을 기억하지 못한다는 것을 의미합니다.메모리를 제공하려면 체크 포인터를 통과해야합니다.체크 포인터를 통과 할 때 에이전트를 호출 할 때`thread_id`도 전달해야합니다 (따라서 재개 할 스레드/대화를 알고 있습니다).


In [19]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [20]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [21]:
input_message = {"role": "user", "content": "Hi, I'm Bob!"}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Hi, I'm Bob!
================================== Ai Message ==================================

Hello Bob! I'm an AI assistant who can help you search for information using specialized search tools. Is there anything specific you'd like to know about or search for? I'm happy to help you find accurate and up-to-date information on various topics.


In [22]:
input_message = {"role": "user", "content": "What's my name?"}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What's my name?
================================== Ai Message ==================================

Your name is Bob, as you introduced yourself earlier. I can remember information shared within our conversation without needing to search for it.


예 [Langsmith Trace] (https://smith.langchain.com/public/fa73960b-0f7d-4910-b73d-757a12f33b2b/r)


새로운 대화를 시작하려면`thrable_id`를 변경하기 만하면됩니다.


In [23]:
# highlight-next-line
config = {"configurable": {"thread_id": "xyz123"}}

input_message = {"role": "user", "content": "What's my name?"}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What's my name?
================================== Ai Message ==================================

I apologize, but I don't have access to any tools that would tell me your name. I can only assist you with searching for publicly available information using the tavily_search function. I don't have access to personal information about users. If you'd like to tell me your name, I'll be happy to address you by it.


## 결론

그것은 랩입니다!이 빠른 시작에서 우리는 간단한 에이전트를 만드는 방법을 다루었습니다.
그런 다음 중간 단계뿐만 아니라 토큰으로 응답을 되 찾는 방법을 보여주었습니다!
우리는 또한 메모리에 추가하여 대화를 나눌 수 있습니다.
에이전트는 배울 것이 많은 복잡한 주제입니다!

에이전트에 대한 자세한 내용은 [langgraph] (/docs/concepts/architecture/#langgraph) 문서를 확인하십시오.여기에는 자체 개념, 튜토리얼 및 방법 안내서 세트가 있습니다.
